## Model training and registration

In this notebook we will:

- Register a model to the model registry.
- Fetch the model from the model registry.

This will introduce the `hsml` (**H**opsworks **M**achine **L**earning) library, which contains functionality to keep track of models and deploy them.

*Note: you need to enable Serving to run this notebook. Go to the **old** Hopsworks UI, click on settings, and check the Serving box.*

In [1]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


### Load Best Hyperparameters

We will train a model using the best hyperparameters we found in the previous notebook.
Recall that we saved these as a pickled dictionary that we uploaded to our cluster.
If you don't have the file locally you can download it using the *hopsworks* library.

In [2]:
import hopsworks

hopsworks_conn = hopsworks.connection()
project = hopsworks_conn.get_project()
dataset_api = project.get_dataset_api()

uploaded_file_path = "Resources/best_params.pickle"
dataset_api.download(uploaded_file_path)

Connected. Call `.close()` to terminate connection gracefully.


'/srv/hops/staging/private_dirs/d53098136e6355738677dfe1bd0cbac9320ccefcc3f6ed636b22171b31a72481/best_params.pickle'

Retrieve the best hyperparameters from this file.

In [3]:
import pickle

with open("best_params.pickle", "rb") as f:
    best_params = pickle.load(f)

n_estimators = best_params["n_estimators"]

print(n_estimators)

89


### Load Training Data & Train Model

Next, we'll train a model in the same way as the previous notebook.

In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

# Load data.
feature_view = fs.get_feature_view("churn_feature_view", 1)
_, td_df = feature_view.get_training_dataset_splits({'train': 70, 'validation': 30}, start_time=None, end_time=None, version = 1)

X_train = td_df["train"]
X_val = td_df['validation']

# Separate target feature from input features.
target = target = feature_view.label[0]  
y_train = X_train.pop(target)
y_val = X_val.pop(target)

# Train model.
pos_class_weight = 0.9
clf = LogisticRegression(class_weight={0: 1.0 - pos_class_weight, 1: pos_class_weight}, solver='liblinear')
clf.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight={0: 0.09999999999999998, 1: 0.9},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='liblinear',
                   tol=0.0001, verbose=0, warm_start=False)

We will also compute some evaluation metrics, which we will register together with the model.

In [7]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

preds = clf.predict(X_val)

precision, recall, fscore, _ = precision_recall_fscore_support(y_val, preds, average="binary")

metrics = {
    'precision': precision,
    'recall': recall,
    'fscore': fscore
}

print(classification_report(y_val, preds))


              precision    recall  f1-score   support

           0       0.96      0.49      0.65      1556
           1       0.39      0.95      0.55       533

    accuracy                           0.61      2089
   macro avg       0.68      0.72      0.60      2089
weighted avg       0.82      0.61      0.62      2089



### Register model

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [9]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


Before registering the model we will export it as a pickle file using joblib.

In [10]:
import joblib

joblib.dump(clf, 'model.pkl')

['model.pkl']

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [11]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'gender', 'type': 'int64'},
   {'name': 'seniorcitizen', 'type': 'int64'},
   {'name': 'partner', 'type': 'int64'},
   {'name': 'dependents', 'type': 'int64'},
   {'name': 'tenure', 'type': 'float64'},
   {'name': 'phoneservice', 'type': 'int64'},
   {'name': 'multiplelines', 'type': 'int64'},
   {'name': 'internetservice', 'type': 'int64'},
   {'name': 'onlinesecurity', 'type': 'int64'},
   {'name': 'onlinebackup', 'type': 'int64'},
   {'name': 'deviceprotection', 'type': 'int64'},
   {'name': 'techsupport', 'type': 'int64'},
   {'name': 'streamingtv', 'type': 'int64'},
   {'name': 'streamingmovies', 'type': 'int64'},
   {'name': 'contract', 'type': 'int64'},
   {'name': 'paperlessbilling', 'type': 'int64'},
   {'name': 'paymentmethod', 'type': 'int64'},
   {'name': 'monthlycharges', 'type': 'float64'},
   {'name': 'totalcharges', 'type': 'float64'}]},
 'output_schema': {'columnar_schema': [{'name': 'churn', 'type': 'int64'}]}}

With the schema in place, we can finally register our model.

In [12]:
model = mr.sklearn.create_model(
    name="churn_tutorial_model",
    metrics=metrics,
    description=f"Random forest model trained with {n_estimators} trees.",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save('model.pkl')


Exported model churn_tutorial_model with version 1


Model(name: 'churn_tutorial_model', version: 1)

Here we have also saved an input example from the training data, which can be helpful for test purposes.

It's important to know that every time you save a model with the same name, a new version of the model will be saved, so nothing will be overwritten. In this way, you can compare several versions of the same model - or create a model with a new name, if you prefer that.

#### Finding the best performing model

Let's imagine you have trained and registered several versions of the same model. Now you can query the model registry for the best model according to your preferred criterion, say F1-score in our case.

The `direction` option is used to indicate if the metric should be high or low (max or min); in our case, it should be high (max).

In [13]:
best_model = mr.get_best_model(name="churn_tutorial_model", metric="fscore", direction="max")
best_model.to_dict()

{'id': 'churn_tutorial_model_1',
 'experimentId': None,
 'projectName': 'churn',
 'experimentProjectName': 'churn',
 'name': 'churn_tutorial_model',
 'modelSchema': {'href': 'https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/128/dataset/Projects/churn/Models/churn_tutorial_model/1/model_schema.json',
  'zip_state': 'NONE'},
 'version': 1,
 'description': 'Random forest model trained with 89 trees.',
 'inputExample': {'href': 'https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/128/dataset/Projects/churn/Models/churn_tutorial_model/1/input_example.json',
  'zip_state': 'NONE'},
 'framework': 'SKLEARN',
 'metrics': {'precision': '0.3887605850654349',
  'recall': '0.9474671669793621',
  'fscore': '0.5513100436681223'},
 'trainingDataset': None,
 'environment': ['/Projects/churn/Models/churn_tutorial_model/1/environment.yml'],
 'program': 'Models/churn_tutorial_model/1/program.ipynb'}

### Next Steps

In the next notebook, we'll look at model serving for the model we just registered to the Model Registry.